### Dataframes from Original DART API CSVs

In [856]:
import pandas as pd 
import numpy as np 

bs = pd.read_csv('dart_bs.csv')
cis = pd.read_csv('dart_cis.csv')
cf = pd.read_csv('dart_cf.csv')
incs = pd.read_csv('dart_incs.csv') 

In [857]:
bs['label_en'] = bs['label_en'].str.lower()
cis['label_en'] = cis['label_en'].str.lower()
cf['label_en'] = cf['label_en'].str.lower()
incs['label_en'] = incs['label_en'].str.lower()

### Merge Comprehensive Income Statement with Income Statement 

In [858]:
corp_with_incs = incs['corp'].unique()

for corp in corp_with_incs: 
    right_index = cis[cis['corp'] == corp].index[-1] 
    
    cis_left = cis.iloc[:right_index + 1]
    cis_right  = cis.iloc[right_index + 1:] 
    
    incs_rows = incs[incs['corp'] == corp]
    
    cis = pd.concat([cis_left, incs_rows, cis_right], ignore_index=True) 

In [10]:
cis.to_csv('cis_updated.csv', encoding='utf-8-sig', index=False) 

### Get KOSPI List from DART API 

In [859]:
import dart_fss as dart
api_key='58b7a0538b393d6c96f7f93b31101d5f407c9d1d'
dart.set_api_key(api_key=api_key)

corp_list = dart.get_corp_list()
kospi_list = corp_list.find_by_corp_name(corp_name='', market='Y')

### EDA 
- 산업별로 그래프 

- null값, 이상치, 분포, 산포

In [ ]:
years = ['2018', '2019', '2020', '2021', '2022'] 

print(bs[years].isna().sum(), end="\n\n")
print(cis[years].isna().sum(), end="\n\n")
print(cf[years].isna().sum())

### USEFUL FUNCTIONS 

In [861]:
def process_feature(df, prefix):
    
    years = ['2018', '2019', '2020', '2021', '2022']

    for year in years:
        original_column = f"{prefix}_{year}"
        new_column = year
        df.rename(columns={original_column: new_column}, inplace=True)
    
    final_df = pd.melt(df, id_vars=['corp'], value_vars=years, var_name='year', value_name=prefix)
    
    final_df = final_df.sort_values(by=['corp', 'year']).reset_index(drop=True)

    return final_df

def find_missing_corps(main_df, subset_df):
    main_corp_unique = set(main_df['corp'].unique())
    subset_corp_unique = set(subset_df['corp'].unique())

    missing_corps = main_corp_unique - subset_corp_unique
    
    return missing_corps

In [904]:
def feature_operation(cis, company_name, feature_name, feat1, feat2, operation, years=['2018', '2019', '2020', '2021', '2022']):
    
    feature_1 = cis.loc[(cis['label_en'] == feat1) & (cis['corp'] == company_name), years]
    feature_2 = cis.loc[(cis['label_en'] == feat2) & (cis['corp'] == company_name), years]
    
    result_list = []
    
    for year in years:
        values1 = feature_1[year].values
        values2 = feature_2[year].values

        if values1.size > 0 and values2.size > 0:
            if operation == 'divide':
                result = values1[0] / values2[0]
            elif operation == 'add':
                result = values1[0] + values2[0]
            else:
                result = np.nan
        else:
            result = np.nan
        
        result_list.append({'corp': company_name, 'year': year, feature_name: result})
    
    result_df = pd.DataFrame(result_list)
    
    return result_df

### Operating Income

In [905]:

operating_income = cis[cis['label_en'].isin((
    'operating income(loss)', 
    'sales account', 
    'operating loss', 
    'operating income', 
    'operating profit', 
    'operating profit (loss)',
    'operating profit(loss)',
    'operating profits'
))]

operating_income = operating_income.drop_duplicates(subset='corp', keep='first')

find_missing_corps(cis, operating_income) 

feature_operating_income = process_feature(operating_income.copy(), 'operating_income') 

feature_operating_income = feature_operating_income.reset_index(drop=True)

### Revenue

In [907]:

revenue_list = ['revenue(sales)', 'revenue', 'sales(revenue)', 'sales', 'sales of goods', 'net sales', 'sales income', 'sales account']

revenue = cis[cis['label_en'].isin((
    revenue_list
))]

revenue = revenue.drop_duplicates(subset='corp', keep='first')

feature_revenue = process_feature(revenue.copy(), 'revenue') 

find_missing_corps(cis, revenue) 

{'LG유플러스', 'LX홀딩스', '대웅', '아이에이치큐', '카카오', '케이티스카이라이프'}

In [908]:
revenue1 = feature_operation(cis, 'LG유플러스', 'revenue', 'operating income', 'operating expenses', 'divide')
revenue2 = feature_operation(cis, 'LX홀딩스', 'revenue', 'operating income', 'operating expenses', 'divide')
revenue3 = feature_operation(cis, '대웅', 'revenue', 'duddjqtndlr', 'selling general administrative expenses', 'add')
revenue4 = feature_operation(cis, '아이에이치큐', 'revenue', 'gross profit', 'selling general administrative expenses', 'add')
revenue5 = feature_operation(cis, '카카오', 'revenue', 'gross profit', 'selling general administrative expenses', 'add')
revenue6 = feature_operation(cis, '케이티스카이라이프', 'revenue', 'operating revenues', 'operating expenses', 'add')

In [ ]:
revenue1 = feature_operation(cis, 'LG유플러스', 'revenue', 'operating income', 'operating expenses', 'divide', years)
revenue2 = feature_operation(cis, 'LX홀딩스', 'revenue', 'operating income', 'operating expenses', 'divide', years)
revenue3 = feature_operation(cis, '대웅', 'revenue', 'duddjqtndlr', 'selling general administrative expenses', 'add', years)
revenue4 = feature_operation(cis, '아이에이치큐', 'revenue', 'gross profit', 'selling general administrative expenses', 'add', years)
revenue5 = feature_operation(cis, '카카오', 'revenue', 'gross profit', 'selling general administrative expenses', 'add', years)
revenue6 = feature_operation(cis, '케이티스카이라이프', 'revenue', 'operating revenues', 'operating expenses', 'add', years)

for revenue in [revenue1, revenue2, revenue3, revenue4, revenue5, revenue6]: 
    feature_revenue = feature_revenue.append(revenue) 

feature_revenue = feature_revenue.reset_index(drop=True)

feature_revenue = feature_revenue.drop_duplicates(subset=['corp', 'year'], keep='first')

### Net Income

In [953]:
net_income_list = ['profit', 'profit (loss)', 'profit(loss)', 'profit (loss) for the period', 'profit(loss) for the period', 'profit (loss) for the year',
'loss (profit)', 'net income(loss)', 'net income', 'profit for the year', 'quarterly net income, attributable to', 'semiannual net profit',
'net profit during thr current term', 'the year net profit(loss)', 'net profit', 'ifrs_profitloss', 'quarterly net profit', '- profit (loss)']

net_income = cis[cis['label_en'].isin((
    net_income_list
))]

net_income = net_income.drop_duplicates(subset='corp', keep='first')

net_income = process_feature(net_income.copy(), 'net_income')

find_missing_corps(cis, net_income) 

{'남해화학', '일진전기'}

In [954]:
mask = (cis['corp'] == '일진전기') & (cis['label_en'] == 'profit (loss), attributable to non-controlling interests')
cis.loc[mask, '2018'] = 0
cis.loc[mask, '2019'] = 0
cis.loc[mask, '2020'] = 0
cis.loc[mask, '2021'] = 0
cis.loc[mask, '2022'] = 0

In [957]:
net_income_1 = feature_operation(cis, '남해화학', 'net_income', 'profit (loss), attributable to owners of parent', 'profit (loss), attributable to non-controlling interests', 'add', years)
net_income_2 = feature_operation(cis, '일진전기', 'net_income', 'profit (loss), attributable to owners of parent', 'profit (loss), attributable to non-controlling interests', 'add', years)

feature_net_income = net_income.append([net_income_1, net_income_2])

/var/folders/_r/qrlg6jnd1l5b_xdkq486q0zr0000gn/T/ipykernel_3034/2446638349.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  feature_net_income = net_income.append([net_income_1, net_income_2])


### Number of Outstanding Shares 

In [188]:
api_key='58b7a0538b393d6c96f7f93b31101d5f407c9d1d'
dart.set_api_key(api_key=api_key)

corp_list = dart.get_corp_list()
kospi_list = corp_list.find_by_corp_name(corp_name='', market='Y')

successful_list = list(bs['corp'].unique())
kospi_list = [comp.corp_name for comp in kospi_list]
failed_list = [comp for comp in kospi_list if comp not in successful_list]

In [108]:
import requests

corp_dict = {} 

kospi_list 

for comp in kospi_list: 
    corp_dict[comp.corp_name] = comp.corp_code 

for key in failed_list:
    if key in corp_dict:
        del corp_dict[key]

url = 'https://opendart.fss.or.kr/api/stockTotqySttus.json'
api_key = '58b7a0538b393d6c96f7f93b31101d5f407c9d1d'
years = ['2018', '2019', '2020', '2021', '2022']

results = []

for corp_name, corp_code in corp_dict.items():
    for year in years:
        params = {
            'crtfc_key': api_key,
            'corp_code': corp_code,
            'bsns_year': year,
            'reprt_code': '11011'
        }

        response = requests.get(url, params=params)

        if response.status_code == 200:
            data = response.json()

            total_outstanding_shares = 0
            for entry in data.get('list', []):
                outstanding_shares = entry.get('istc_totqy', '0').replace(',', '')
                if outstanding_shares.isdigit():
                    total_outstanding_shares += int(outstanding_shares)
                    
            results.append({
                'company_name': corp_name,
                'year': year,
                'total_outstanding_shares': total_outstanding_shares
            })
            
        else:
            print(f'Error: Unable to fetch data for {corp_name} in {year}. HTTP Status code: {response.status_code}')

df = pd.DataFrame(results)

df = df.pivot(index='company_name', columns='year', values='total_outstanding_shares')

df.reset_index(inplace=True)

In [949]:
outstanding_shares = pd.read_csv('outstanding_shares.csv')

In [ ]:
feature_outstanding_shares = pd.melt(outstanding_shares, id_vars=["company_name"], var_name="year", value_name="outstanding_shares")
feature_outstanding_shares = feature_outstanding_shares.rename(columns={'company_name': 'corp'})

### Total Liabilities

In [960]:
total_liabilities = bs[bs['label_en']=='total liabilities'] 

total_liabilities.drop_duplicates(subset='corp', keep='first')

find_missing_corps(bs, total_liabilities)

feature_total_liabilities = process_feature(total_liabilities.copy(), 'total_liabilities') 

feature_total_liabilities = feature_total_liabilities.reset_index(drop=True) 

In [961]:
feature_total_liabilities

,corp,year,total_liabilities
0,AJ네트웍스,2018,2.083331e+12
1,AJ네트웍스,2019,1.455914e+12
2,AJ네트웍스,2020,1.300163e+12
3,AJ네트웍스,2021,9.925344e+11
4,AJ네트웍스,2022,1.119560e+12
...,...,...,...
3260,흥아해운,2018,7.305503e+11
3261,흥아해운,2019,4.000433e+11
3262,흥아해운,2020,3.130598e+11
3263,흥아해운,2021,1.443589e+11


### Total Assets

In [962]:
total_assets = bs[bs['label_en']=='total assets']  

total_assets.drop_duplicates(subset='corp', keep='first')

find_missing_corps(bs, total_assets)

feature_total_assets = process_feature(total_assets.copy(), 'total_assets')

feature_total_assets = feature_total_assets.reset_index(drop=True) 

### Cash Flow Per Share

In [1054]:
cfps_list = ['cash flows from (used in) operating activities', 'cash flows from operating activities', 'operating activity cash flow', 'net cash flows from operating activities', 'operating activities', 'i. cash flows from (used in) operating activities']

cfps = cf[cf['label_en'].isin((
    cfps_list
))]

cfps = cfps.drop_duplicates(subset='corp', keep='first')

find_missing_corps(cf, cfps)

feature_cash_flow = process_feature(cfps.copy(), 'cash_flow')

feature_cash_flow.reset_index(drop=True, inplace=True)

In [1041]:
merged_df = pd.merge(feature_cash_flow, feature_outstanding_shares, on=['corp', 'year'])

merged_df['cash_flow_per_share'] = merged_df['cash_flow'] / merged_df['outstanding_shares']

feature_cash_flow_per_share = merged_df.drop(columns=['cash_flow', 'outstanding_shares']) 

### Sales Per Share

In [980]:
merged_df = pd.merge(feature_revenue, feature_outstanding_shares, on=['corp', 'year'])

merged_df['sales_per_share'] = merged_df['revenue'] / merged_df['outstanding_shares']

feature_sales_per_share = merged_df[['corp', 'year', 'sales_per_share']]

### Current Assets

In [984]:
current_assets = bs[bs['label_en']=='current assets'].drop_duplicates(subset='corp', keep='first') 

feature_current_assets = process_feature(current_assets.copy(), 'current_assets') 
feature_current_assets = feature_current_assets.reset_index(drop=True) 

### Current Liabilities

In [986]:
current_liabilities = bs[bs['label_en']=='current liabilities'].drop_duplicates(subset='corp', keep='first') 

feature_current_liabilities = process_feature(current_liabilities.copy(), 'current_liabilities') 
feature_current_liabilities = feature_current_liabilities.reset_index(drop=True) 

### Inventory

In [ ]:
inventories_list = ['inventories']

inventories = bs[bs['label_en'].isin((
    inventories_list
))]

inventories = inventories.drop_duplicates(subset='corp', keep='first')

inventories = process_feature(inventories.copy(), 'inventory')

find_missing_corps(bs, inventories) 


In [992]:
corps = list(find_missing_corps(bs, inventories))
years = ['2018', '2019', '2020', '2021', '2022']

data = []
for corp in corps:
    for year in years:
        data.append({'corp': corp, 'year': year, 'inventory': 0})

inventory_df = pd.DataFrame(data)

feature_inventory = pd.concat([inventories, inventory_df], ignore_index=True)

### Net Working Capital

In [ ]:
merged_df = pd.merge(feature_revenue, feature_outstanding_shares, on=['corp', 'year'])

merged_df['sales_per_share'] = merged_df['revenue'] / merged_df['outstanding_shares']

feature_sales_per_share = merged_df[['corp', 'year', 'sales_per_share']]

In [ ]:
merged_df = pd.merge(feature_current_assets, feature_current_liabilities, on=['corp', 'year'])

merged_df['net_working_capital'] = merged_df['current_assets'] - merged_df['current_liabilities'] 

feature_net_working_capital = merged_df[['corp', 'year', 'net_working_capital']] 

feature_net_working_capital


### Non-current Assets 

In [999]:
non_current_assets_list = ['non-current assets']

non_current_assets = bs[bs['label_en'].isin((
    non_current_assets_list
))]

non_current_assets = non_current_assets.drop_duplicates(subset='corp', keep='first')

non_current_assets = process_feature(non_current_assets.copy(), 'non_current_assets')

feature_non_current_assets = non_current_assets.reset_index(drop=True)

### Non-current Liabilities 

In [1001]:
non_current_liabilities_list = ['non-current liabilities']

non_current_liabilities = bs[bs['label_en'].isin((
    non_current_liabilities_list
))]

non_current_liabilities = non_current_liabilities.drop_duplicates(subset='corp', keep='first')

non_current_liabilities = process_feature(non_current_liabilities.copy(), 'non_current_liabilities')

feature_non_current_liabilities = non_current_liabilities.reset_index(drop=True)

### Combine features 

In [1015]:
feature_list = [feature_operating_income, feature_revenue, feature_net_income,
                feature_outstanding_shares, feature_total_liabilities,
                feature_total_assets, feature_cash_flow_per_share,
                feature_sales_per_share, feature_current_assets,
                feature_current_liabilities, feature_inventory,
                feature_net_working_capital, feature_non_current_assets,
                feature_non_current_liabilities] 

feature_combined = feature_list[0]

for feature_df in feature_list[1:]:
    feature_combined = pd.merge(feature_combined, feature_df, on=['corp', 'year'])
    
feature_combined.replace([np.inf, -np.inf], np.nan, inplace=True)

In [1017]:
feature_combined.to_csv('feature_combined.csv', encoding='utf-8-sig', index=False)

### Feature Combined 전처리

In [ ]:
def feature_operation(cis, company_name, feature_name, feat1, feat2, operation, years=['2018', '2019', '2020', '2021', '2022']):
    
    # Filter the rows for the given company and features
    feature_1 = cis.loc[(cis['label_en'] == feat1) & (cis['corp'] == company_name), years]
    feature_2 = cis.loc[(cis['label_en'] == feat2) & (cis['corp'] == company_name), years]
    
    # Initialize result list
    result_list = []
    
    # Iterate through each year and perform the specified operation
    for year in years:
        values1 = feature_1[year].values
        values2 = feature_2[year].values

        # Check if values are present for both features in the given year
        if values1.size > 0 and values2.size > 0:
            # Perform the specified operation
            if operation == 'divide':
                result = values1[0] / values2[0]
            elif operation == 'add':
                result = values1[0] + values2[0]
            else:
                result = np.nan
        else:
            result = np.nan
        
        # Add the result to the list
        result_list.append({'corp': company_name, 'year': year, feature_name: result})
    
    # Convert the result list to a DataFrame
    result_df = pd.DataFrame(result_list)
    
    return result_df

### Operating Income

In [ ]:

operating_income = cis[cis['label_en'].isin((
    'operating income(loss)', 
    'sales account', 
    'operating loss', 
    'operating income', 
    'operating profit', 
    'operating profit (loss)',
    'operating profit(loss)',
    'operating profits'
))]

operating_income = operating_income.drop_duplicates(subset='corp', keep='first')

find_missing_corps(cis, operating_income) 

feature_operating_income = process_feature(operating_income.copy(), 'operating_income') 

feature_operating_income = feature_operating_income.reset_index(drop=True)

### Revenue

In [ ]:

revenue_list = ['revenue(sales)', 'revenue', 'sales(revenue)', 'sales', 'sales of goods', 'net sales', 'sales income', 'sales account']

revenue = cis[cis['label_en'].isin((
    revenue_list
))]

revenue = revenue.drop_duplicates(subset='corp', keep='first')

feature_revenue = process_feature(revenue.copy(), 'revenue') 

find_missing_corps(cis, revenue) 

{'LG유플러스', 'LX홀딩스', '대웅', '아이에이치큐', '카카오', '케이티스카이라이프'}

In [ ]:
revenue1 = feature_operation(cis, 'LG유플러스', 'revenue', 'operating income', 'operating expenses', 'divide')
revenue2 = feature_operation(cis, 'LX홀딩스', 'revenue', 'operating income', 'operating expenses', 'divide')
revenue3 = feature_operation(cis, '대웅', 'revenue', 'duddjqtndlr', 'selling general administrative expenses', 'add')
revenue4 = feature_operation(cis, '아이에이치큐', 'revenue', 'gross profit', 'selling general administrative expenses', 'add')
revenue5 = feature_operation(cis, '카카오', 'revenue', 'gross profit', 'selling general administrative expenses', 'add')
revenue6 = feature_operation(cis, '케이티스카이라이프', 'revenue', 'operating revenues', 'operating expenses', 'add')

In [ ]:
revenue1 = feature_operation(cis, 'LG유플러스', 'revenue', 'operating income', 'operating expenses', 'divide', years)
revenue2 = feature_operation(cis, 'LX홀딩스', 'revenue', 'operating income', 'operating expenses', 'divide', years)
revenue3 = feature_operation(cis, '대웅', 'revenue', 'duddjqtndlr', 'selling general administrative expenses', 'add', years)
revenue4 = feature_operation(cis, '아이에이치큐', 'revenue', 'gross profit', 'selling general administrative expenses', 'add', years)
revenue5 = feature_operation(cis, '카카오', 'revenue', 'gross profit', 'selling general administrative expenses', 'add', years)
revenue6 = feature_operation(cis, '케이티스카이라이프', 'revenue', 'operating revenues', 'operating expenses', 'add', years)

for revenue in [revenue1, revenue2, revenue3, revenue4, revenue5, revenue6]: 
    feature_revenue = feature_revenue.append(revenue) 

feature_revenue = feature_revenue.reset_index(drop=True)

feature_revenue = feature_revenue.drop_duplicates(subset=['corp', 'year'], keep='first')

### Net Income

In [ ]:
net_income_list = ['profit', 'profit (loss)', 'profit(loss)', 'profit (loss) for the period', 'profit(loss) for the period', 'profit (loss) for the year',
'loss (profit)', 'net income(loss)', 'net income', 'profit for the year', 'quarterly net income, attributable to', 'semiannual net profit',
'net profit during thr current term', 'the year net profit(loss)', 'net profit', 'ifrs_profitloss', 'quarterly net profit', '- profit (loss)']

net_income = cis[cis['label_en'].isin((
    net_income_list
))]

net_income = net_income.drop_duplicates(subset='corp', keep='first')

net_income = process_feature(net_income.copy(), 'net_income')

find_missing_corps(cis, net_income) 

{'남해화학', '일진전기'}

In [ ]:
mask = (cis['corp'] == '일진전기') & (cis['label_en'] == 'profit (loss), attributable to non-controlling interests')
cis.loc[mask, '2018'] = 0
cis.loc[mask, '2019'] = 0
cis.loc[mask, '2020'] = 0
cis.loc[mask, '2021'] = 0
cis.loc[mask, '2022'] = 0

In [ ]:
net_income_1 = feature_operation(cis, '남해화학', 'net_income', 'profit (loss), attributable to owners of parent', 'profit (loss), attributable to non-controlling interests', 'add', years)
net_income_2 = feature_operation(cis, '일진전기', 'net_income', 'profit (loss), attributable to owners of parent', 'profit (loss), attributable to non-controlling interests', 'add', years)

feature_net_income = net_income.append([net_income_1, net_income_2])

### Number of Outstanding Shares 

In [ ]:
api_key='58b7a0538b393d6c96f7f93b31101d5f407c9d1d'
dart.set_api_key(api_key=api_key)

corp_list = dart.get_corp_list()
kospi_list = corp_list.find_by_corp_name(corp_name='', market='Y')

successful_list = list(bs['corp'].unique())
kospi_list = [comp.corp_name for comp in kospi_list]
failed_list = [comp for comp in kospi_list if comp not in successful_list]

In [ ]:
import requests

corp_dict = {} 

kospi_list 

for comp in kospi_list: 
    corp_dict[comp.corp_name] = comp.corp_code 

for key in failed_list:
    if key in corp_dict:
        del corp_dict[key]

url = 'https://opendart.fss.or.kr/api/stockTotqySttus.json'
api_key = '58b7a0538b393d6c96f7f93b31101d5f407c9d1d'
years = ['2018', '2019', '2020', '2021', '2022']

results = []

for corp_name, corp_code in corp_dict.items():
    for year in years:
        params = {
            'crtfc_key': api_key,
            'corp_code': corp_code,
            'bsns_year': year,
            'reprt_code': '11011'
        }

        response = requests.get(url, params=params)

        if response.status_code == 200:
            data = response.json()

            total_outstanding_shares = 0
            for entry in data.get('list', []):
                outstanding_shares = entry.get('istc_totqy', '0').replace(',', '')
                if outstanding_shares.isdigit():
                    total_outstanding_shares += int(outstanding_shares)
                    
            results.append({
                'company_name': corp_name,
                'year': year,
                'total_outstanding_shares': total_outstanding_shares
            })
            
        else:
            print(f'Error: Unable to fetch data for {corp_name} in {year}. HTTP Status code: {response.status_code}')

df = pd.DataFrame(results)

df = df.pivot(index='company_name', columns='year', values='total_outstanding_shares')

df.reset_index(inplace=True)

In [ ]:
outstanding_shares = pd.read_csv('outstanding_shares.csv')

In [ ]:
feature_outstanding_shares = pd.melt(outstanding_shares, id_vars=["company_name"], var_name="year", value_name="outstanding_shares")
feature_outstanding_shares = feature_outstanding_shares.rename(columns={'company_name': 'corp'})

### Total Liabilities

In [ ]:
total_liabilities = bs[bs['label_en']=='total liabilities'] 

total_liabilities.drop_duplicates(subset='corp', keep='first')

find_missing_corps(bs, total_liabilities)

feature_total_liabilities = process_feature(total_liabilities.copy(), 'total_liabilities') 

feature_total_liabilities = feature_total_liabilities.reset_index(drop=True) 

### Total Assets

In [ ]:
total_assets = bs[bs['label_en']=='total assets']  

total_assets.drop_duplicates(subset='corp', keep='first')

find_missing_corps(bs, total_assets)

feature_total_assets = process_feature(total_assets.copy(), 'total_assets')

feature_total_assets = feature_total_assets.reset_index(drop=True) 

### Cash Flow Per Share

In [ ]:
cfps_list = ['cash flows from (used in) operating activities', 'cash flows from operating activities', 'operating activity cash flow', 'net cash flows from operating activities', 'operating activities', 'i. cash flows from (used in) operating activities', 'cash flows from (used in) operating activities.', 'net cash flow from operating activities', 'cash flows provided by operating activities', 'cash flows from operating activities', 'net cash flows from used in operations']

cfps = cf[cf['label_en'].isin(cfps_list)]

cfps['nonnull_count'] = cfps[['2018', '2019', '2020', '2021', '2022']].count(axis=1)

cfps.sort_values('nonnull_count', ascending=False, inplace=True)

cfps.drop_duplicates(subset='corp', keep='first', inplace=True)

find_missing_corps(cf, cfps)

feature_cash_flow = process_feature(cfps.copy(), 'cash_flow')

feature_cash_flow.reset_index(drop=True, inplace=True)

In [1179]:
merged_df = pd.merge(feature_cash_flow, feature_outstanding_shares, on=['corp', 'year'])

merged_df['cash_flow_per_share'] = merged_df['cash_flow'] / merged_df['outstanding_shares']

feature_cash_flow_per_share = merged_df.drop(columns=['cash_flow', 'outstanding_shares']) 

### Sales Per Share

In [ ]:
merged_df = pd.merge(feature_revenue, feature_outstanding_shares, on=['corp', 'year'])

merged_df['sales_per_share'] = merged_df['revenue'] / merged_df['outstanding_shares']

feature_sales_per_share = merged_df[['corp', 'year', 'sales_per_share']]

### Current Assets

In [ ]:
current_assets = bs[bs['label_en']=='current assets'].drop_duplicates(subset='corp', keep='first') 

feature_current_assets = process_feature(current_assets.copy(), 'current_assets') 
feature_current_assets = feature_current_assets.reset_index(drop=True) 

### Current Liabilities

In [ ]:
current_liabilities = bs[bs['label_en']=='current liabilities'].drop_duplicates(subset='corp', keep='first') 

feature_current_liabilities = process_feature(current_liabilities.copy(), 'current_liabilities') 
feature_current_liabilities = feature_current_liabilities.reset_index(drop=True) 

### Inventory

In [ ]:
inventories_list = ['inventories']

inventories = bs[bs['label_en'].isin((
    inventories_list
))]

inventories = inventories.drop_duplicates(subset='corp', keep='first')

inventories = process_feature(inventories.copy(), 'inventory')

find_missing_corps(bs, inventories) 


In [ ]:
corps = list(find_missing_corps(bs, inventories))
years = ['2018', '2019', '2020', '2021', '2022']

data = []
for corp in corps:
    for year in years:
        data.append({'corp': corp, 'year': year, 'inventory': 0})

inventory_df = pd.DataFrame(data)

feature_inventory = pd.concat([inventories, inventory_df], ignore_index=True)

### Net Working Capital

In [ ]:
merged_df = pd.merge(feature_current_assets, feature_current_liabilities, on=['corp', 'year'])

merged_df['net_working_capital'] = merged_df['current_assets'] - merged_df['current_liabilities'] 

feature_net_working_capital = merged_df[['corp', 'year', 'net_working_capital']] 


### Non-current Assets 

In [ ]:
non_current_assets_list = ['non-current assets']

non_current_assets = bs[bs['label_en'].isin((
    non_current_assets_list
))]

non_current_assets = non_current_assets.drop_duplicates(subset='corp', keep='first')

non_current_assets = process_feature(non_current_assets.copy(), 'non_current_assets')

feature_non_current_assets = non_current_assets.reset_index(drop=True)

### Non-current Liabilities 

In [ ]:
non_current_liabilities_list = ['non-current liabilities']

non_current_liabilities = bs[bs['label_en'].isin((
    non_current_liabilities_list
))]

non_current_liabilities = non_current_liabilities.drop_duplicates(subset='corp', keep='first')

non_current_liabilities = process_feature(non_current_liabilities.copy(), 'non_current_liabilities')

feature_non_current_liabilities = non_current_liabilities.reset_index(drop=True)

### Combine features 

In [1329]:
feature_list = [feature_operating_income, feature_revenue, feature_net_income,
                feature_outstanding_shares, feature_total_liabilities,
                feature_total_assets, feature_cash_flow_per_share,
                feature_sales_per_share, feature_current_assets,
                feature_current_liabilities, feature_inventory,
                feature_net_working_capital, feature_non_current_assets,
                feature_non_current_liabilities] 

feature_combined = feature_list[0]

for feature_df in feature_list[1:]:
    feature_combined = pd.merge(feature_combined, feature_df, on=['corp', 'year'])
    
feature_combined.replace([np.inf, -np.inf], np.nan, inplace=True)

In [1330]:
feature_combined.shape

(3265, 16)

### 전처리 how? 

1. 2022 없으면 그 corp다 삭제 

2. 2021 & 2022는 있는데 2018, 2019, 2020 없음 -> 그냥 2018 2019 2020 row 제거 

3. 이렇게 하고 나면 30 Rows 남는데 대부분이 발행주식수(outstanding_shares) 데이터가 없어서 cash_flow_per_share & sales_per_share 값이 null값임 -> 제거 

4. 나머지 8 Rows는 operating income, revenue, net income, inventory, assets 이런 데이터가 없음. 3000+ rows중에 8개라서 그냥 없어도 분석에는 지장이 없음 -> 제거 



In [1331]:
mask = (feature_combined['year'] == '2022') & (feature_combined.isnull().any(axis=1))

null_2022 = feature_combined.loc[mask]

corps_to_remove = null_2022['corp'].tolist()

print(f"{len(corps_to_remove)} corps have been removed since there's no relevant data for 2022")

feature_combined = feature_combined[~feature_combined['corp'].isin(corps_to_remove)]

21 corps have been removed since there's no relevant data for 2022


In [1332]:
feature_combined.shape

(3160, 16)

In [1333]:
len(feature_combined['corp'].unique())

632

In [1334]:
null_rows = feature_combined[feature_combined.isnull().any(axis=1)]

In [1335]:
mask = null_rows.isnull().sum(axis=1) >= 5

corps_and_years_to_remove = null_rows.loc[mask, ['corp', 'year']].values.tolist()

original_corp_len = len(feature_combined['corp'].unique()) 

for corp, year in corps_and_years_to_remove:
    mask = (feature_combined['corp'] == corp) & (feature_combined['year'] == year)
    feature_combined = feature_combined[~mask]

In [1336]:
feature_combined.shape

(3087, 16)

In [1288]:
feature_combined.to_csv('feature_combined.csv', encoding='utf-8-sig', index=False)
null_rows = feature_combined[feature_combined.isnull().any(axis=1)]
null_rows.to_csv('null_rows.csv', encoding='utf-8-sig', index=False)

In [1337]:
null_rows = feature_combined[feature_combined.isnull().any(axis=1)]

In [1346]:
corps_and_years_to_remove = null_rows[['corp', 'year']].values.tolist()

for corp, year in corps_and_years_to_remove:
    mask = (feature_combined['corp'] == corp) & (feature_combined['year'] == year)
    feature_preprocessed = feature_combined[~mask]

In [1347]:
feature_preprocessed.shape

(3057, 16)

In [1348]:
feature_preprocessed.to_csv('feature_preprocessed.csv', encoding='utf-8-sig', index=False)